In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def searchForNoiseNYPD(row):
    if(row['Agency'] == 'NYPD'):
        complaint_type = row['Complaint Type']
        if('Noise' not in complaint_type):
            row['Complaint Type'] = np.nan
    return row

In [4]:
dFrame = pd.read_csv('../../rawdata/311/311.csv')

/home/jlrulff/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8,17,31,32,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
dFrame = dFrame[['Created Date', 'Closed Date', 'Agency', 'Agency Name', 'Complaint Type', 'Descriptor', 'Latitude', 'Longitude']]

In [9]:
dFrame = dFrame[ (dFrame['Agency'] == 'NYPD') | ((dFrame['Agency'] == 'DEP') & (dFrame['Complaint Type'] == 'Noise'))]
dFrame = dFrame.apply(lambda row: searchForNoiseNYPD(row), axis=1)
dFrame = dFrame.dropna(subset=['Complaint Type'])
dFrame = dFrame[['Created Date', 'Closed Date', 'Descriptor', 'Latitude', 'Longitude']]
dFrame = dFrame.dropna()

In [12]:
dFrame['datetime'] = pd.to_datetime(dFrame['Created Date'])
dFrame = dFrame.set_index(['datetime'])
dFrame.sort_index(inplace=True)

In [1]:
dFrame.head()

NameError: name 'dFrame' is not defined

In [ ]:
dFrame.to_pickle('../../data/311/311.pkl', index=False)